In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from google.colab.patches import cv2_imshow
from google.colab import files
from tensorflow.keras import layers, models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
# Load the CSV file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get the file paths
pb_files = '/content/drive/MyDrive/Deep learning lab6/opencv_face_detector_uint8.pb'
pbtxt_files = '/content/drive/MyDrive/Deep learning lab6/opencv_face_detector.pbtxt'
h5_files = '/content/drive/MyDrive/Deep learning lab6/inception_v3_weights_tf_dim_ordering_tf_kernels.h5'



In [ ]:
# Load the OpenCV face detector model
net = cv2.dnn.readNetFromTensorflow(pb_files, pbtxt_files)


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# Function to perform face detection on a static photo
def detect_faces(frame):
    global net

    # Perform face detection
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104., 177., 123.], False, False)
    net.setInput(blob)
    detections = net.forward()

    # Process the detections
    for i in range(0, detections.shape[2]):
        # Get the confidence (probability) of the current detection:
        confidence = detections[0, 0, i, 2]
        # Only consider detections if confidence is greater than a fixed minimum confidence:
        if confidence > 0.7:
            # Get the coordinates of the current detection:
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")
            # Draw the detection and the confidence:
            text = "{:.3f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 0, 0), 3)
            cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Display the result
    cv2_imshow(frame)


In [ ]:
def detect_and_crop_faces(image_path):
    global net
    frame = cv2.imread(image_path)

    # Perform face detection
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104., 177., 123.], False, False)
    net.setInput(blob)
    detections = net.forward()

    # Initialize variables to store the coordinates of the face if found
    face_coords = None

    # Process the detections
    for i in range(0, detections.shape[2]):
        # Get the confidence (probability) of the current detection:
        confidence = detections[0, 0, i, 2]

        # Only consider detections if confidence is greater than a fixed minimum confidence:
        if confidence > 0.7:
            # Get the coordinates of the current detection:
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")

            # Update face_coords with the coordinates of the current detection
            face_coords = (startX, startY, endX, endY)

            # Break out of the loop as soon as one face is found
            break

    # If face_coords is None, it means no face was found, so return None
    if face_coords is None:
        return None

    # Otherwise, crop the face region using the coordinates and return it
    (startX, startY, endX, endY) = face_coords
    face = frame[startY:endY, startX:endX]
    return face


In [ ]:
import cv2
import numpy as np
from IPython.display import Image
from PIL import Image as PIL_Image
from io import BytesIO

try:
    # Capture the photo
    filename = take_photo()
    print('Saved to {}'.format(filename))

    # Load the image using PIL
    img_pil = PIL_Image.open(filename)

    # Convert PIL Image to NumPy array
    img_np = np.array(img_pil)

    # Show the image which was just taken.
    display(Image(filename))

except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))


In [ ]:
detect_faces(img_np)

In [ ]:
# Load images and labels from Google Drive
drive_path = '/content/drive/My Drive/Deep learning lab6/105_classes_pins_dataset'
output_path = '/content/drive/My Drive/Deep learning lab6/cropped_images'


# Cropping Dataset Images

In [ ]:
# Preprocess images and detect faces separately
if not os.path.exists(output_path):
        os.makedirs(output_path)
        print("'cropped_images' directory created")
c=1
for subdir in os.listdir(drive_path):
    print(c)
    print("\n")
    c=c+1
    subdir_path = os.path.join(drive_path, subdir)
    if os.path.isdir(subdir_path):
        output_subdir_path = os.path.join(output_path, subdir)
        print(output_subdir_path)
        if not os.path.exists(output_subdir_path):
            os.makedirs(output_subdir_path)
        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)
            if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
                face = detect_and_crop_faces(file_path)
                if face is not None:
                    output_file_path = os.path.join(output_subdir_path, file)
                    cv2.imwrite(output_file_path, face)

# Spliting Dataset

In [ ]:
# Define paths
train_dir = "/content/drive/My Drive/Deep learning lab6/train"
val_dir = "/content/drive/My Drive/Deep learning lab6/val"
test_dir = "/content/drive/My Drive/Deep learning lab6/test"

In [ ]:
# Create directories for subsets
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
c=1
# Split dataset into training, validation, and testing sets
for person_dir in os.listdir(output_path):
    print(c)
    c=c+1
    print(person_dir)

    # Create subdirectories in train_dir, val_dir, and test_dir
    os.makedirs(os.path.join(train_dir, person_dir), exist_ok=True)
    os.makedirs(os.path.join(val_dir, person_dir), exist_ok=True)
    os.makedirs(os.path.join(test_dir, person_dir), exist_ok=True)

    person_images = os.listdir(os.path.join(output_path, person_dir))
    train_images, test_val_images = train_test_split(person_images, test_size=0.3, random_state=42)
    val_images, test_images = train_test_split(test_val_images, test_size=0.5, random_state=42)

    # Move images to respective directories
    for image in train_images:
        shutil.copy(os.path.join(output_path, person_dir, image), os.path.join(train_dir, person_dir, image))
    for image in val_images:
        shutil.copy(os.path.join(output_path, person_dir, image), os.path.join(val_dir, person_dir, image))
    for image in test_images:
        shutil.copy(os.path.join(output_path, person_dir, image), os.path.join(test_dir, person_dir, image))
